# 어텐션(Attention)

### 맥락 벡터 : 단어 벡터에 가중치를  곱하여 합한 가중합을 구한 벡터
단어를 선택하는 작업은 미분 불가하므로 모든 것을 선택하고 단어의 중요도를 가중치로 계산

In [13]:
import numpy as np
np.random.seed(100)

T, H = 5, 4                # T : 시계열의 길이, H : Hidden size
hs = np.random.randn(T,H)  # (5,4)

print('hs:\n',hs)
print('hs0:\n',hs[0])

a = np.array([0.8, 0.1, 0.03, 0.05, 0.02]) # 가중치 , (5,)
print('a:\n',a)

# (1) repeat() 함수 사용
ar = a.reshape(T,1).repeat(4,axis=1) # (5,1)로 2차원으로 shape을 바꾸고 수평 방향으로 4번 반복 복사
print('ar:\n',ar)                    # (5,4)

t = hs * ar                          # (5,4) * (5,4) : 요소간의 곱셈, 단어벡터에 가중치를 곱함
print('t:\n',t)
print(t.shape)                       # (5,4)

# 가중합
c = np.sum(t,axis=0)                 # 수직 방향으로 합 , 가중합, 맥락 벡터
print('c  :',c)
print('hs0:',hs[0])
print(c.shape)                       # (4,)

print('-'*60)

# (2) 브로드캐스팅 사용, 1번과 결과 동일 
ar = a.reshape(T,1)                  # (5,1)로 2차원으로 shape을 바꿈
print('ar:\n',ar)                    # (5,1)

t = hs * ar                          # (5,4) * (5,1) : 브로드캐스팅 적용, , 단어벡터에 가중치를 곱함
print('t:\n',t)
print(t.shape)                       # (5,4)           

# 가중합
c = np.sum(t,axis=0)
print('c  :',c)
print('hs0:',hs[0])
print(c.shape)


hs:
 [[-1.74976547  0.3426804   1.1530358  -0.25243604]
 [ 0.98132079  0.51421884  0.22117967 -1.07004333]
 [-0.18949583  0.25500144 -0.45802699  0.43516349]
 [-0.58359505  0.81684707  0.67272081 -0.10441114]
 [-0.53128038  1.02973269 -0.43813562 -1.11831825]]
hs0:
 [-1.74976547  0.3426804   1.1530358  -0.25243604]
a:
 [0.8  0.1  0.03 0.05 0.02]
ar:
 [[0.8  0.8  0.8  0.8 ]
 [0.1  0.1  0.1  0.1 ]
 [0.03 0.03 0.03 0.03]
 [0.05 0.05 0.05 0.05]
 [0.02 0.02 0.02 0.02]]
t:
 [[-1.39981238  0.27414432  0.92242864 -0.20194883]
 [ 0.09813208  0.05142188  0.02211797 -0.10700433]
 [-0.00568487  0.00765004 -0.01374081  0.0130549 ]
 [-0.02917975  0.04084235  0.03363604 -0.00522056]
 [-0.01062561  0.02059465 -0.00876271 -0.02236636]]
(5, 4)
c  : [-1.34717053  0.39465326  0.95567913 -0.32348518]
hs0: [-1.74976547  0.3426804   1.1530358  -0.25243604]
(4,)
------------------------------------------------------------
ar:
 [[0.8 ]
 [0.1 ]
 [0.03]
 [0.05]
 [0.02]]
t:
 [[-1.39981238  0.27414432  0.92242864 

In [25]:
# 3차원 배열의 가중합의 구현, 맥락 벡터
np.random.seed(100)

N,T,H = 10,5,4             # 면,행,열
hs = np.random.randn(N,T,H)
# print('hs:\n',hs)

np.random.seed(100)
a = np.random.randn(N,T)   # 가중치의 합이 1이 아닌 예임
print('a:\n',a)

ar = a.reshape(N,T,1).repeat(H,axis=2)  # 2번축(열)로 4번 반복  , repeat()함수는 생략 가능
# print('ar:\n',ar)

t = hs * ar    # (10,5,4)
print(t.shape)
# print('t:\n',t)

c = np.sum(t, axis=1)   # 1번 축(행)으로 합 , (10,4)
print(c.shape)
# print('c:\n',c)
# print('hs0:\n',hs[:,0,:])

a:
 [[-1.74976547  0.3426804   1.1530358  -0.25243604  0.98132079]
 [ 0.51421884  0.22117967 -1.07004333 -0.18949583  0.25500144]
 [-0.45802699  0.43516349 -0.58359505  0.81684707  0.67272081]
 [-0.10441114 -0.53128038  1.02973269 -0.43813562 -1.11831825]
 [ 1.61898166  1.54160517 -0.25187914 -0.84243574  0.18451869]
 [ 0.9370822   0.73100034  1.36155613 -0.32623806  0.05567601]
 [ 0.22239961 -1.443217   -0.75635231  0.81645401  0.75044476]
 [-0.45594693  1.18962227 -1.69061683 -1.35639905 -1.23243451]
 [-0.54443916 -0.66817174  0.00731456 -0.61293874  1.29974807]
 [-1.73309562 -0.9833101   0.35750775 -1.6135785   1.47071387]]
(10, 5, 4)
(10, 4)


#### 가중합 WeightSum 계층 구현 : 맥락벡터를 구하는 계층

In [26]:
# 가중합을 구하는 class
class WeightSum:
    def __init__(self):
        self.params, self.grads = [],[]   # 학습 매개변수가 없는 계층
        self.cache = None
        
    def forward(self, hs, a):
        N, T, H = hs.shape
        
        ar = a.reshape(N,T,1)#.repeat(T,axis=2)  # repeat노드이지만 repeat 사용하지 않아도 아래 연산시 브로드캐스팅 적용
        t = hs * ar
        c = np.sum(t, axis = 1)
        
        self.cache = (hs,ar)
        return c
    
    def backward(self,dc):
        hs,ar = self.cache
        N, T, H = hs.shape
        dt = dc.reshape(N,1,H).repeat(T,axis=1)  # sum의 역전파, 출력: (N,T,H)
        dar = dt * hs            # (N,T,H)      
        dhs = dt * ar            # (N,T,H)
        da = np.sum(dar,axis=2)  # repeat의 역전파, 출력 :(N,T) 
        
        return dhs, da

### 가중치를 구하는 AttentionWeight 계층 구현
: 각 단어의 가중치를 구하여 WeightSum 계층으로 전달한다

In [31]:
# 가중치 a를 softmax를 사용하여 구하기
from nn_layers import softmax, Softmax

# nn_layers.py에 아래 Softmax class추가
# class Softmax:
#     def __init__(self):
#         self.params, self.grads = [], []
#         self.out = None

#     def forward(self, x):
#         self.out = softmax(x)
#         return self.out

#     def backward(self, dout):
#         dx = self.out * dout
#         sumdx = np.sum(dx, axis=1, keepdims=True)
#         dx -= self.out * sumdx
#         return dx

N,T,H = 10,5,4
hs = np.random.randn(N,T,H)
h  = np.random.randn(N,H)

hr = h.reshape(N,1,H).repeat(T,axis=1)

t = hs * hr
print(t.shape)        # (N,T,H)

s = np.sum(t,axis=2)  # (N,T)
print(s.shape)

soft_max = Softmax()
a = soft_max.forward(s)
print(a.shape)
print(a.sum(axis=1))
# print(a)

(10, 5, 4)
(10, 5)
(10, 5)
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[[1.87699824e-01 3.60040601e-01 2.95807623e-01 1.02205102e-01
  5.42468494e-02]
 [8.77836348e-03 9.89047271e-02 2.04681743e-01 1.70273125e-02
  6.70607854e-01]
 [7.69312219e-04 1.67733625e-01 2.52945212e-01 9.01172184e-04
  5.77650678e-01]
 [6.40883592e-04 2.44121891e-01 6.13086618e-02 6.87572097e-01
  6.35646637e-03]
 [3.33814015e-01 1.07726889e-01 3.21857785e-02 2.17159379e-01
  3.09113938e-01]
 [6.68132725e-01 2.46531507e-02 1.22167333e-01 1.36481273e-01
  4.85655175e-02]
 [5.76386728e-01 8.52571395e-02 1.64464901e-01 2.32384904e-02
  1.50652741e-01]
 [6.33386626e-01 3.92542682e-02 9.58075785e-02 3.38453453e-02
  1.97706182e-01]
 [7.00447207e-02 1.44515422e-02 3.60745020e-01 2.21424090e-02
  5.32616308e-01]
 [5.07140413e-03 3.03548242e-01 3.78220628e-03 1.00228739e-03
  6.86595861e-01]]
